In [28]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from pathlib import Path
import shutil

In [29]:
def removeIdentifierFile(path):
    for file in os.listdir(path):
        if file.find(".Identifier") != -1:
            os.remove(os.path.join(path, file))

In [30]:
import os, random

raw_imposters_folder = "new_imposter"
raw_valid_user_folder = "valid_user"

removeIdentifierFile(raw_imposters_folder)
removeIdentifierFile(raw_valid_user_folder)

img_count = 500 # img taken from each class

imposter_filenames = random.sample(os.listdir(raw_imposters_folder), img_count)
valid_user_filenames = random.sample(os.listdir(raw_valid_user_folder), img_count)

train_valid_user_folder = "train/valid_user"
train_imposters_folder = "train/imposters"

Path(train_valid_user_folder).mkdir(parents=True, exist_ok=True)
Path(train_imposters_folder).mkdir(parents=True, exist_ok=True)

test_valid_user_folder = "test/valid_user"
test_imposters_folder = "test/imposters"

Path(test_valid_user_folder).mkdir(parents=True, exist_ok=True)
Path(test_imposters_folder).mkdir(parents=True, exist_ok=True)


In [31]:
class_training_size = int(img_count * 0.9)

count = 0
for imposter_filename in imposter_filenames[:class_training_size]:
    shutil.copy(os.path.join(raw_imposters_folder, imposter_filename), os.path.join(train_imposters_folder, imposter_filename))
    count += 1

count = 0
for imposter_filename in imposter_filenames[class_training_size:]:
    shutil.copy(os.path.join(raw_imposters_folder, imposter_filename), os.path.join(test_imposters_folder, imposter_filename))


for valid_user_filename in valid_user_filenames[:class_training_size]:
    shutil.copy(os.path.join(raw_valid_user_folder, valid_user_filename), os.path.join(train_valid_user_folder, valid_user_filename))

for valid_user_filename in valid_user_filenames[class_training_size:]:
    shutil.copy(os.path.join(raw_valid_user_folder, valid_user_filename), os.path.join(test_valid_user_folder, valid_user_filename))

removeIdentifierFile(train_imposters_folder)
removeIdentifierFile(test_imposters_folder)
removeIdentifierFile(train_valid_user_folder)
removeIdentifierFile(test_valid_user_folder)


In [32]:
train = ImageDataGenerator(rescale=1/255)
test = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory("train",
    target_size=(150,150),
    batch_size = 32,
    class_mode = 'binary')
                                         
test_dataset = test.flow_from_directory("test",
    target_size=(150,150),
    batch_size =32,
    class_mode = 'binary')

Found 1130 images belonging to 2 classes.
Found 193 images belonging to 2 classes.


In [33]:
test_dataset.class_indices


{'imposters': 0, 'valid_user': 1}

In [34]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(64,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Conv2D(128,(3,3),activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(512,activation='relu'))

model.add(keras.layers.Dense(1,activation='sigmoid'))

In [35]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [36]:
model.fit_generator(train_dataset,
    steps_per_epoch = 250,
    epochs = 10,
    validation_data = test_dataset
)

model.save('classifier')

Epoch 1/10


/tmp/ipykernel_1448/1593974248.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_dataset,


  1/250 [..............................] - ETA: 9:25 - loss: 0.7091 - accuracy: 0.4375

KeyboardInterrupt: 

In [ ]:
def predictImage(filename):
    img1 = image.load_img(filename,target_size=(150,150))
    model = tf.keras.models.load_model('classifier')
 
    Y = image.img_to_array(img1)
    
    X = np.expand_dims(Y,axis=0)
    val = model.predict(X, verbose=0)
    return val[0][0]




In [37]:
test_imposter = "test/imposters"
test_valid_user = "test/valid_user"
correct_count = 0
wrong_count = 0
count = 0

for file in os.listdir(test_imposter):
    id = predictImage(os.path.join(test_imposter, file))
    count += 1
    if id == 0:
        correct_count += 1
        print(f"{file} correct {id}")
    else:
        wrong_count += 1
        print(f"{file} wrong {id}")


for file in os.listdir(test_valid_user):
    id = predictImage(os.path.join(test_valid_user, file))
    count += 1
    if id == 1:
        correct_count += 1
        print(f"{file} correct {id}")
    else:
        wrong_count += 1
        print(f"{file} wrong {id}")

340.png correct 0.0
209.png correct 0.0
333.png correct 0.0
416.png wrong 1.0
420.png correct 0.0
317.png correct 0.0
449.png correct 0.0
369.png correct 0.0
360.png correct 0.0
435.png correct 0.0
302.png correct 0.0
175.png correct 0.0
53.png correct 0.0
442.png correct 0.0
576.png correct 0.0
472.png correct 0.0
536.png correct 0.0
176.png correct 0.0
299.png correct 0.0
305.png correct 0.0
30.png correct 0.0
192.png correct 0.0
270.png correct 0.0
481.png correct 0.0
497.png correct 0.0
264.png correct 0.0
110.png correct 0.0
495.png correct 0.0
390.png correct 0.0
407.png correct 0.0
118.png correct 0.0
297.png correct 0.0
251.png correct 0.0
131.png correct 0.0
321.png correct 0.0
221.png correct 0.0
184.png correct 0.0
491.png correct 0.0
483.png correct 0.0
227.png correct 0.0
553.png correct 0.0
464.png correct 0.0
540.png correct 0.0
239.png correct 0.0
412.png correct 0.0
386.png correct 0.0
365.png correct 0.0
149.png correct 0.0
447.png correct 0.0
50.png correct 0.0
432.p

In [38]:
print(correct_count / count)


0.9948186528497409
